# 18강. 딥러닝 레이어의 이해(1) Linear, Convolution

## Linear Layer

In [1]:
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size, 4, 2)) # 사각형 2개 세트를 batch_size개만큼 만든 후 처리

print("1단계 연산 준비:", boxes.shape)

first_linear = tf.keras.layers.Dense(units=1, use_bias=False)
# units은 출력 차원 수
# Weight 행렬 속 실수를 인간의 뇌 속 하나의 뉴런 유닛 취급

first_out = first_linear(boxes)
first_out = tf.squeeze(first_out, axis=-1) # (4, 1)을 (4,)로 변환

print("1단계 연산 결과:", first_out.shape)
print("1단계 Linear Layer의 Weight 형태:", first_linear.weights[0].shape)

print("\n2단계 연산 준비:", first_out.shape)

second_linear = tf.keras.layers.Dense(units=1, use_bias=False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out, axis=-1)

print('2단계 연산 결과:', second_out.shape)
print('2단계 Linear Layer의 Weight 형태:', second_linear.weights[0].shape)

1단계 연산 준비: (64, 4, 2)
1단계 연산 결과: (64, 4)
1단계 Linear Layer의 Weight 형태: (2, 1)

2단계 연산 준비: (64, 4)
2단계 연산 결과: (64,)
2단계 Linear Layer의 Weight 형태: (4, 1)


In [2]:
batch_size = 64
boxes = tf.zeros((batch_size, 4, 2)) # (4,2)차원

print('1단계 연산 준비:', boxes.shape)

# (4,3)차원으로 확장
first_linear = tf.keras.layers.Dense(units=3, use_bias=False)
first_out = first_linear(boxes)

print('1단계 연산 결과:', first_out.shape)
print('1단계 Linear Layer의 Weight 형태:', first_linear.weights[0].shape)

print('\n2단계 연산 준비:', first_out.shape)

# Dense = Linear
second_linear = tf.keras.layers.Dense(units=1, use_bias=False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out, axis=-1)

print('2단계 연산 결과:', second_out.shape)
print('2단계 Linear Layer의 Weight 형태:', second_linear.weights[0].shape)

print('\n3단계 연산 준비:', second_out.shape)

# 4차원을 하나의 실수로 집약
third_linear = tf.keras.layers.Dense(units=1, use_bias=False)
third_out = third_linear(second_out)
third_out = tf.squeeze(third_out, axis=-1)

print('3단계 연산 결과:', third_out.shape)
print('3단계 Linear Layer의 Weight 형태:', third_linear.weights[0].shape)

# 모든 params를 더하여 total_params 구하기
total_params = \
first_linear.count_params() + \
second_linear.count_params() + \
third_linear.count_params()

print('총 Parameters:', total_params)

1단계 연산 준비: (64, 4, 2)
1단계 연산 결과: (64, 4, 3)
1단계 Linear Layer의 Weight 형태: (2, 3)

2단계 연산 준비: (64, 4, 3)
2단계 연산 결과: (64, 4)
2단계 Linear Layer의 Weight 형태: (3, 1)

3단계 연산 준비: (64, 4)
3단계 연산 결과: (64,)
3단계 Linear Layer의 Weight 형태: (4, 1)
총 Parameters: 13


![그림설명](https://user-images.githubusercontent.com/116326867/203595589-8e036605-72fb-4205-a0e6-7901d88bb4e7.png)

## Convolution Layer

In [3]:
batch_size = 64
pic = tf.zeros((batch_size, 1920, 1080, 3)) # 폭 1920, 높이 1080, 채널 3(컬러)

print('입력 이미지 데이터:', pic.shape)
conv_layer = tf.keras.layers.Conv2D(filters=16,
                                   kernel_size=(5, 5),
                                   strides=5,
                                   use_bias=False)
conv_out = conv_layer(pic)

print("\nConvolution 결과:", conv_out.shape)
print("Convolution Layer의 Parameter 수:", conv_layer.count_params())

flatten_out = tf.keras.layers.Flatten()(conv_out)
print("\n1차원으로 펼친 데이터:", flatten_out.shape)

linear_layer = tf.keras.layers.Dense(units=1, use_bias=False)
linear_out = linear_layer(flatten_out)

print('\nLinear 결과:', linear_out.shape)
print('Linear Layer의 Parameter 수:', linear_layer.count_params())

입력 이미지 데이터: (64, 1920, 1080, 3)

Convolution 결과: (64, 384, 216, 16)
Convolution Layer의 Parameter 수: 1200

1차원으로 펼친 데이터: (64, 1327104)

Linear 결과: (64, 1)
Linear Layer의 Parameter 수: 1327104


입력이미지데이터(배치사이즈, 폭, 높이, 채널수)  
→ Convolution(배치사이즈, 폭/stride, 높이/stride, 필터수)  
→ Convolution Layer의 파라미터(채널수 x 필터수 x 커널사이즈(a x b))  
→ 1차원으로 바꿔주기  
→ Linear 변환  
→ Linear Layer의 파라미터(폭/stride x 높이/stride x 필터수)